In [1]:
%%capture

!pip install /kaggle/input/pytorch-geometric-packages/torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl
!pip install /kaggle/input/pytorch-geometric-packages/torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl
!pip install torch-geometric
!pip install ogb

In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GINEConv, global_mean_pool
from torch_geometric.data import DataLoader
from torch_geometric.datasets import QM9
from torch_geometric.transforms import NormalizeFeatures
import torch_geometric
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch.nn import (
    BatchNorm1d,
    Embedding,
    Linear,
    ModuleList,
    ReLU,
    Sequential,
)
from typing import Any, Dict, Optional

# The following based on the official example of GPSConv: 
https://github.com/pyg-team/pytorch_geometric/blob/master/examples/graph_gps.py#L34

In [3]:
class RedrawProjection:
    def __init__(self, model: torch.nn.Module,
                 redraw_interval: Optional[int] = None):
        self.model = model
        self.redraw_interval = redraw_interval
        self.num_last_redraw = 0

    def redraw_projections(self):
        if not self.model.training or self.redraw_interval is None:
            return
        if self.num_last_redraw >= self.redraw_interval:
            fast_attentions = [
                module for module in self.model.modules()
                if isinstance(module, PerformerAttention)
            ]
            for fast_attention in fast_attentions:
                fast_attention.redraw_projection_matrix()
            self.num_last_redraw = 0
            return
        self.num_last_redraw += 1

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import BatchNorm, GPSConv

class GPS(torch.nn.Module):
    def __init__(self, channels: int, pe_dim: int, num_layers: int,
                 attn_type: str, attn_kwargs: Dict[str, Any]):
        super().__init__()

        self.node_emb = Embedding(28, channels - pe_dim)
        self.pe_lin = Linear(3, pe_dim)
        self.pe_norm = BatchNorm1d(3)
        self.edge_emb = Embedding(4, channels)

        self.convs = ModuleList()
        for _ in range(num_layers):
            nn = Sequential(
                Linear(channels, channels),
                ReLU(),
                Linear(channels, channels),
            )
            conv = GPSConv(channels, GINEConv(nn), heads=4,
                           attn_type=attn_type, attn_kwargs=attn_kwargs)
            self.convs.append(conv)

        self.mlp = Sequential(
            Linear(channels, channels // 2),
            ReLU(),
            Linear(channels // 2, channels // 4),
            ReLU(),
            Linear(channels // 4, 1),
        )
        self.redraw_projection = RedrawProjection(
            self.convs,
            redraw_interval=1000 if attn_type == 'performer' else None)

    def forward(self, x, pe, edge_index, edge_attr, batch):
        x_pe = self.pe_norm(pe)
        # Convert node features to Long tensor for embedding lookup
        x = x.squeeze(-1).long()  # Ensure x is treated as indices
        x = torch.cat((self.node_emb(x), self.pe_lin(x_pe)), 1)
        edge_attr = self.edge_emb(edge_attr)

        for conv in self.convs:
            x = conv(x, edge_index, batch, edge_attr=edge_attr)
        x = global_add_pool(x, batch)
        return self.mlp(x)




In [5]:
dataset_name = 'ogbn-arxiv'
# transform = T.AddRandomWalkPE(walk_length=20, attr_name='pe')
transform = T.AddRandomWalkPE(walk_length=3, attr_name='pe')

dataset = PygNodePropPredDataset(name=dataset_name,
                                 transform=transform)
# dataset = PygNodePropPredDataset(name=dataset_name,
#                                  transform=T.ToSparseTensor())

data = dataset[0]

# Make the adjacency matrix to symmetric
# data.adj_t = data.adj_t.to_symmetric()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# If you use GPU, the device should be cuda
print('Device: {}'.format(device))

data = data.to(device)
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:01<00:00, 56.69it/s]
Processing...

Extracting dataset/arxiv.zip


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 8594.89it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 343.20it/s]

Saving...



Done!
/opt/conda/lib/python3.10/site-packages/ogb/nodeproppred/dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.pro

Device: cuda


In [6]:
# def train(model, data, train_idx, optimizer, loss_fn):
#     model.train()
#     loss = 0

#     optimizer.zero_grad()
# #     model.redraw_projection.redraw_projections()
#     out = model(data.x, data.pe, data.edge_index, data.edge_attr,
#                     data.batch)
#     loss = loss_fn(out[train_idx], data.y[train_idx].squeeze(1))
#     #########################################

#     loss.backward()
#     optimizer.step()

#     return loss.item()

In [6]:
# Test function here
@torch.no_grad()
def test(model, data, split_idx, evaluator, save_model_results=False):
    # TODO: Implement a function that tests the model by
    # using the given split_idx and evaluator.
    model.eval()

    # The output of model on all data
    out = None
    out = model(data.x, data.pe, data.edge_index, data.edge_attr,
                    data.batch)

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    if save_model_results:
      print ("Saving Model Predictions")

      data = {}
      data['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

      df = pd.DataFrame(data=data)
      # Save locally as csv
      df.to_csv('ogbn-arxiv_node.csv', sep=',', index=False)


    return train_acc, valid_acc, test_acc

In [7]:
# Please do not change the args

args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
    'num_heads': 2,  
}
args

{'device': 'cuda',
 'num_layers': 3,
 'hidden_dim': 256,
 'dropout': 0.5,
 'lr': 0.01,
 'epochs': 100,
 'num_heads': 2}

In [8]:
attn_kwargs = {'dropout': 0.5}
model = GPS(channels=64, pe_dim=8, num_layers=2, attn_type='multihead',
            attn_kwargs=attn_kwargs).to(device)

In [10]:
evaluator = Evaluator(name='ogbn-arxiv')

In [17]:
# import copy

# optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
# loss_fn = F.nll_loss

# best_model = None
# best_valid_acc = 0

# for epoch in range(1, 1 + args["epochs"]):
#     loss = train(model, data, train_idx, optimizer, loss_fn)
#     result = test(model, data, split_idx, evaluator)
#     train_acc, valid_acc, test_acc = result
#     if valid_acc > best_valid_acc:
#         best_valid_acc = valid_acc
#         best_model = copy.deepcopy(model)
#     print(f'Epoch: {epoch:02d}, '
#           f'Loss: {loss:.4f}, '
#           f'Train: {100 * train_acc:.2f}%, '
#           f'Valid: {100 * valid_acc:.2f}% '
#           f'Test: {100 * test_acc:.2f}%')

In [12]:
torch.cuda.empty_cache()


In [13]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [9]:
batch_size = 8

In [10]:
def train(model, data, train_idx, optimizer, loss_fn, batch_size):
    model.train()
    total_loss = 0
    # Shuffle the training indices
    permutation = torch.randperm(len(train_idx))
    for i in range(0, len(train_idx), batch_size):
        optimizer.zero_grad()
        indices = train_idx[permutation[i:i+batch_size]]
        # Extract batch inputs and labels
        batch_x = data.x[indices]
        batch_y = data.y[indices]
#         output = model(batch_x)
#         model.redraw_projection.redraw_projections()
        out = model(data.x, data.pe, data.edge_index, data.edge_attr,
                    data.batch)
        loss = loss_fn(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        print("iter")
        print(len(indices))
    return total_loss / (len(train_idx) / batch_size)


In [11]:
import copy

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = F.nll_loss

best_model = None
best_valid_acc = 0

In [12]:
for epoch in range(1, 2):
    loss = train(model, data, train_idx, optimizer, loss_fn, batch_size)
#     result = test(model, data, split_idx, evaluator)
#     train_acc, valid_acc, test_acc = result
#     if valid_acc > best_valid_acc:
#         best_valid_acc = valid_acc
#         best_model = copy.deepcopy(model)
#     print(f'Epoch: {epoch:02d}, '
#           f'Loss: {loss:.4f}, '
#           f'Train: {100 * train_acc:.2f}%, '
#           f'Valid: {100 * valid_acc:.2f}% '
#           f'Test: {100 * test_acc:.2f}%')


/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [46,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [46,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [46,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [46,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [46,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [46,0,0], thread: [5,0,0] Assertion `srcIndex < 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
